In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

In [2]:
data_regress = pd.read_csv('D:\python\data\kc_house_data3.csv')

In [3]:
data_classification = pd.read_csv('D:\python\data\card_transdata4.csv')

## Задание 1

In [4]:
data_regress.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
data_regress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14934 entries, 0 to 14933
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             14934 non-null  int64  
 1   date           14934 non-null  int64  
 2   price          14934 non-null  int64  
 3   bedrooms       14934 non-null  int64  
 4   bathrooms      14934 non-null  float64
 5   sqft_living    14934 non-null  int64  
 6   sqft_lot       14934 non-null  int64  
 7   floors         14934 non-null  int64  
 8   waterfront     14934 non-null  int64  
 9   view           14934 non-null  int64  
 10  condition      14934 non-null  int64  
 11  grade          14934 non-null  int64  
 12  sqft_above     14934 non-null  int64  
 13  sqft_basement  14934 non-null  int64  
 14  yr_built       14934 non-null  int64  
 15  yr_renovated   14934 non-null  int64  
 16  zipcode        14934 non-null  int64  
 17  lat            14934 non-null  float64
 18  long  

In [6]:
y_reg = data_regress["price"]
X_reg = data_regress.drop(["price"], axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_reg, y_reg, test_size=0.2)

In [8]:
inputs = tf.keras.Input(shape=(20,))
x = tf.keras.layers.Dense(64, activation="relu")(inputs) #Dense - полносвязный слой
x = tf.keras.layers.Dense(32, activation="linear")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(16, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1, activation="linear")(x)
model_regression = tf.keras.Model(inputs=inputs, outputs=outputs)


In [9]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mae") 

In [10]:
model_regression.fit(X_train, y_train, epochs=150, verbose=None)

In [11]:
y_pred = model_regression.predict(X_test)

94/94 [==============================] - 0s 893us/step


In [12]:
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

139552.39977402077
-0.09033718218664544


## Classification

In [13]:
data_classification.drop('Unnamed: 0', axis=1, inplace=True)

In [20]:
class_0 = data_classification[data_classification['fraud'] == 0]
class_1 = data_classification[data_classification['fraud'] == 1]
class_0_downsampled = resample(class_0, replace=False, n_samples=int(0.1 * len(class_0)), random_state=42)# Уменьшаем размер подвыборки для класса 0
small_data_classification = pd.concat([class_0_downsampled, class_1])# Объединяем данные обратно
data_classification = small_data_classification.sample(frac=1, random_state=42)# Перемешиваем данные, чтобы сохранить случайность

In [21]:
data_classification.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13088 entries, 252968 to 194398
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   distance_from_home              13088 non-null  float64
 1   distance_from_last_transaction  13088 non-null  float64
 2   ratio_to_median_purchase_price  13088 non-null  float64
 3   repeat_retailer                 13088 non-null  float64
 4   used_chip                       13088 non-null  float64
 5   used_pin_number                 13088 non-null  float64
 6   online_order                    13088 non-null  float64
 7   fraud                           13088 non-null  float64
dtypes: float64(8)
memory usage: 920.2 KB


In [22]:
y_classifier = data_classification["fraud"]
X_classifier = data_classification.drop(["fraud"], axis=1)

In [23]:
X_train_classifier, X_test_classifier, y_train_classifier, y_test_classifier = train_test_split(X_classifier, y_classifier, test_size=0.3)

In [24]:
inputs = tf.keras.Input(shape=(7,))
x = tf.keras.layers.Dense(128, activation="relu")(inputs) #Dense - полносвязный слой
x = tf.keras.layers.Dense(64, activation="linear")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
outputs = tf.keras.layers.Dense(1, activation="linear")(x)
model_class = tf.keras.Model(inputs=inputs, outputs=outputs)


In [25]:
model_class.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss="mae") 

In [26]:
model_class.fit(X_train_classifier, y_train_classifier, epochs=10, verbose=None)

In [27]:
y_pred = model_class.predict(X_test_classifier, verbose=None)

In [33]:
scaler = MinMaxScaler()

y_pred_scaled = scaler.fit_transform(y_pred.reshape(-1, 1))

y_pred_scaled = y_pred_scaled.flatten()

rounded_predictions = np.round(y_pred_scaled)

In [34]:
print(classification_report(y_test_classifier, rounded_predictions))

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98      1663
         1.0       0.97      1.00      0.99      2264

    accuracy                           0.98      3927
   macro avg       0.99      0.98      0.98      3927
weighted avg       0.98      0.98      0.98      3927



## MLP

In [35]:
import numpy as np

class MLP:
    def __init__(self, layer_sizes, activation_functions):
        self.num_layers = len(layer_sizes)
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i-1]) for i in range(1, self.num_layers)]
        self.biases = [np.random.randn(layer_sizes[i], 1) for i in range(1, self.num_layers)]

    def _sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-x))

    def _sigmoid_derivative(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))

    def _tanh(self, x):
        return np.tanh(x)

    def _tanh_derivative(self, x):
        return 1 - np.tanh(x)**2

    def _relu(self, x):
        return np.maximum(0, x)

    def _relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def _feed_forward(self, x):
        # х-ый элеменет
        activations = [x]
        zs = []
        for i in range(self.num_layers - 1):
            weight = self.weights[i]
            bias = self.biases[i]
            activation_fn = self.activation_functions[i]
            #умножаем матрицу весов на вектор активаций последнего слоя + вектор смещения
            z = np.dot(weight, activations[-1]) + bias
            zs.append(z)
            if activation_fn == 'sigmoid':
                a = self._sigmoid(z)
            elif activation_fn == 'tanh':
                a = self._tanh(z)
            elif activation_fn == 'relu':
                a = self._relu(z)
            activations.append(a)
        return activations, zs

    def _backpropagate(self, x, y):
        # накопление градиентов функции потерь по весам и смещениям в каждом слое сети
        delta_weights = [np.zeros(weight.shape) for weight in self.weights]
        delta_biases = [np.zeros(bias.shape) for bias in self.biases]
        # прямой проход сети для входных данных x, и сохраняются активации и взвешенные суммы для каждого слоя 
        activations, zs = self._feed_forward(x)
        #разница между предсказанными активациями и фактическими значениями целевой переменной
        delta = (activations[-1] - y) 
        for i in range(self.num_layers - 2, -1, -1):
            # Получение взвешенной суммы
            z = zs[i]
            activation_fn = self.activation_functions[i]
            if activation_fn == 'sigmoid':
                derivative = self._sigmoid_derivative(z)
            elif activation_fn == 'tanh':
                derivative = self._tanh_derivative(z)
            elif activation_fn == 'relu':
                derivative = self._relu_derivative(z)
            #Вычисление градиента по весам для текущего слоя
            delta_weights[i] = np.dot(delta, activations[i].T)
            delta_biases[i] = delta
            delta = np.dot(self.weights[i].T, delta) * derivative
        return delta_weights, delta_biases

    def fit(self, X_train, y_train, learning_rate, num_epochs):
        for epoch in range(num_epochs):
            for x, y in zip(X_train, y_train):
                x = np.array(x, ndmin=2).T
                y = np.array(y, ndmin=2).T
                delta_weights, delta_biases = self._backpropagate(x, y)
                self.weights = [weight - learning_rate * d_weight for weight, d_weight in zip(self.weights, delta_weights)]
                self.biases = [bias - learning_rate * d_bias for bias, d_bias in zip(self.biases, delta_biases)]

    def predict(self, X_test):
        predictions = []
        for x in X_test:
            # массив минимум 2мерный и транспонируем
            x = np.array(x, ndmin=2).T
            activations, _ = self._feed_forward(x)
            predictions.append(activations[-1])
        return predictions

In [36]:
X_train_regression = np.asarray(X_train, dtype=np.float64)
y_train_regression = np.asarray(y_train, dtype=np.float64).reshape(-1, 1)

mlp_regression = MLP([20, 1, 1], ['sigmoid', 'sigmoid'])

# Обучаем модель
mlp_regression.fit(X_train_regression, y_train_regression, learning_rate=0.05, num_epochs=10)

X_test_regression = np.asarray(X_test, dtype=np.float64)
# Получаем предсказания модели
predictions_regression = mlp_regression.predict(X_test_regression)

C:\Users\Сабака\AppData\Local\Temp\ipykernel_2076\3666063162.py:12: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [37]:
predictions_regression = np.asarray(predictions_regression, dtype=np.float64).reshape(-1, 1)

In [38]:
y_test_regression = np.asarray(y_test, dtype=np.float64).reshape(-1, 1)
from sklearn.metrics import r2_score
print(mean_absolute_error(y_test_regression, predictions_regression))
print(mean_squared_error(y_test_regression, predictions_regression))
print(r2_score(y_test_regression, predictions_regression))

436396.0060261132
221555161639.6063
-6.120826201761699


In [39]:
# классификация 

In [40]:
X_train_classifier = np.asarray(X_train_classifier, dtype=np.float64)
y_train_classifier = np.asarray(y_train_classifier, dtype=np.float64).reshape(-1, 1)

In [41]:
mlp_classifier = MLP([7, 1, 1, 1], ['tanh', 'relu', 'sigmoid'])

mlp_classifier.fit(X_train_classifier, y_train_classifier, learning_rate=0.05, num_epochs=10)

X_test_classifier = np.asarray(X_test_classifier, dtype=np.float64)
# Получаем предсказания модели
predictions_classifier = mlp_classifier.predict(X_test_classifier)

In [42]:
predictions_classifier = np.asarray(predictions_classifier, dtype=np.float64).reshape(-1, 1)

In [ ]:
rounded_predictions = np.round(predictions_classifier)

In [49]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_classifier,rounded_predictions[:3927])
print("Accuracy:", accuracy)

Accuracy: 0.5765215176979883
